### MineSweeper as Jupyter Noteook

Minesweeper, the funnest game Windows 3.1 had to offer!

Given a number of rows, columns, and mines, design and code a class that implements the core logic of the game.  You can assume there is a UI API already created for you, you just need to be able to say what needs to be displayed at what coordinates.

At the start of the game, the user is presented a [rows x columns] 2D grid of blank cels.  The user clicks a cel, and action occurs depending on what's there:

MINE - Game's over.  Reveal the board and exit.

MINE-ADJACENT - Cels that have one or more mines as an immediate neighbor (including diagonals) reveal as a number, which is the total number of adjacent mines.

NOTHING-NEAR - Cels that have zero adjacent mines reveal themselves as blank and then auto-click all neighboring cels to reveal them as well.  The game knows that every cel surrounding is safe to click, so we save the user the hassle.

The game is won when all cels are uncovered, excepting for the cels containing mines.


In [178]:
debugging = False
debugging = True

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

In [209]:
class Cell():
    
    def __init__(self, row, col, rows, cols):
        self.row  = row
        self.col  = col
        self.rows = rows
        self.cols = cols
        self.mine = False
        self.revealed = False
        
    def pos(self):
        return (self.row, self.col)

    def valid_pos(self, row, col):
        return col>=0 and col<self.cols and row>=0 and row<self.rows
    
    def is_valid(self):
        row, col = self.pos()
        return self.valid_pos(row, col)

    def desc(self):
        row, col = self.pos()
        return '{0}{1},{2}'.format('**' if self.mine else '', row, col)
    
    def surroundings(self):
        ''' Return a list the (up to) nine valid positions around this cell '''
        row, col = self.pos()
        for y in range(row-1, row+2):
            for x in range(col-1, col+2):
                if self.valid_pos(y, x):
                    yield (y, x)

In [210]:
import ipywidgets as widgets
from IPython.display import display
from random import randint

UNKNOWN = '?'
EMPTY = ' '
MINE = '**'

class Board():
    def __init__(self, game, rows, cols, density=10):
        '''
        The density is the integer percent of cells which will probably
        contain mines.
        '''
        assert density > 0 and density < 90
        assert cols < 50 and rows < 50

        self.rows = rows
        self.cols = cols
        self.game = game
        self.density = density
        
        columns = []
        vbox    = []
        for y in range(rows):
            row = []
            for x in range(cols):
                btn = widgets.Button(description=UNKNOWN)
                btn.layout.width = '100px' if debugging else '50px'
                btn.icon = 'none'                
                row.append(btn)
                row[x].on_click(Board.on_cell_button_clicked_event)
                row[x].value = ( Cell(y, x, rows, cols), self )
            vbox.append(widgets.HBox(row))
            columns.append(row)      
        self.board = columns
        self.vbox  = widgets.VBox(vbox)
        self.plant_landmines()
        self.count_all_cells()

    def show(self):
        display(self.vbox) 

    def button(self, pos):
        ''' Get the button that describes this positon '''
        row, col = pos
        return self.board[row][col]
    
    def cell(self, pos):
        ''' Get the cell that describes this positon '''
        button = self.button(pos)
        return button.value[0]
        
    def all_cell_positions(self):
        ''' Generate a list of all cells on the board '''
        for row in range(self.rows):
            for col in range(self.cols):
                yield (row, col)
                
    def plant_landmines(self):
        ''' Mark a random set of cells as having been mined. '''
        mines = 0
        self.mines_marked_correctly = 0
        for pos in self.all_cell_positions():
            cell = self.cell(pos)
            mine = randint(0,99) < self.density
            mines += 1 if mine else 0
        if mines == 0:
            # Always plant at least one mine.
            row = randint(0, self.rows-1)
            col = randint(0, self.cols-1)
            self.cell((row, col)).mine = True
            mines = 1
        self.mines = mines

    def count_mines(self, button):
        cell = button.value[0]
        mine_count = 1 if cell.mine else 0
        for rc in cell.surroundings():
            c = self.cell(rc)
            mine_count += 1 if c.mine else 0
        cell.mine_count = mine_count
        cell.revealed = False
        
    def count_all_cells(self):
        ''' Compute the adjacent mines for all cells '''
        for pos in self.all_cell_positions():
            button = self.button(pos)
            self.count_mines(button)
        
    def show_count(self, button, cell):
        ''' Show the count all the mines adjacent to this cell '''
        button.description = EMPTY if cell.mine_count == 0 else str(cell.mine_count)
        if cell.mine:
            button.description = MINE
        cell.revealed = True

    def reveal_empty_cells(self, button, cell):
        ''' Flow the revealed cells outward from this one '''
        if cell.mine_count == 0:
            for rc in cell.surroundings():
                c = self.cell(rc)
                b = self.button(rc)
                if not c.revealed:
                    self.show_count(b, c)
                    self.reveal_empty_cells(b, c)
                    
    def reveal_all_cells(self):
        for pos in self.all_cell_positions():
            button = self.button(pos)
            cell = self.cell(pos)
            self.show_count(button, cell)            
            
    def click_button(self, button, cell):
        ''' Do everything related to a single button click on a cell '''
        if self.game.probing:
            button.icon = 'none'
            self.show_count(button, cell)
            if cell.mine:
                self.game.game_over_man()
                self.reveal_all_cells()
            self.reveal_empty_cells(button, cell)
        else:
            if not cell.revealed:
                if button.icon == 'check':
                    # Uncheck a cell that may not really be a mine.
                    button.icon = 'none'
                    self.mines_marked_correctly -= 1 if cell.mine else 0
                    if debugging or cell.revealed:
                        self.show_count(button, cell)
                    else:
                        button.description = UNKNOWN
                else:
                    button.icon = 'check'
                    button.description = EMPTY
                    self.mines_marked_correctly += 1 if cell.mine else 0
                dbg("Marked {0} of {1} mines correctly", self.mines_marked_correctly, self.mines)               
                if self.mines_marked_correctly == self.mines:
                    self.game.congratulations()
                    self.reveal_all_cells()
        
    @staticmethod
    def on_cell_button_clicked_event(button):
        ''' The cell clicked event handler '''
        cell = button.value[0]
        self = button.value[1]
        self.click_button(button, cell)        

In [211]:
MSG_PROBING = 'Currently in Mine PROBING mode. Be Very Careful!'
MSG_MARKING = 'Currently in Mine MARKING mode.'

class Game():
   
    def __init__(self, rows=9, cols=9, density=20):
        self.board = Board(self, rows, cols, density)
        self.probing = False;
        self.gameover = False        
        
        modeButton = widgets.Button(
            description='Initializing...',
            disabled=False,
            button_style='info', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            width='300px',
            icon='check'
        )
        
        self.modeButton = modeButton

        handler=Game.toggle_mode_button_event
        modeButton.value = self
        modeButton.on_click(handler)
        modeButton.layout.width = '400px'
        handler(modeButton)

    @staticmethod
    def toggle_mode_button_event(btn):
        self = btn.value
        self.probing = not self.probing
        if self.probing:
            btn.description=MSG_PROBING
            btn.icon='none'     
        else:
            btn.description=MSG_MARKING
            btn.icon='check'
            
    def announce(self, f, *args):
        ''' Display a status text line '''
        print((f).format(*args))
        
    def game_over_man(self):
        self.gameover = True
        self.announce("***** KABOOM! *****   You are dead. ")

    def congratulations(self):
        if self.gameover:
            self.announce("I hope you're happy now. You've marked every deadly mine correctly.")
            self.announce("Unfortunately, that seems somewhat pointless. Have a nice day.")              
        else:
            self.gameover = True 
            self.announce("Congratulations! You've marked every deadly mine correctly.")
            self.announce("You are a winner!")  

    def run(self):
        self.announce("Minesweeper! The exciting game of blowing yourself up --- or not!")
        display(self.modeButton)
        self.board.show()
        self.announce("The game is won when you've correctly marked every mine.")


In [212]:
debugging = False
Game(5, 4, 1).run()

Minesweeper! The exciting game of blowing yourself up --- or not!


A Jupyter Widget

A Jupyter Widget

The game is won when you've correctly marked every mine.
Congratulations! You've marked every deadly mine correctly.
You are a winner!
